### Functions and Objects

In [1]:
import yaml
import pandas as pd
from web3 import Web3
import datetime
import numpy as np
import json
import os

# Function to read in YAML file
def read_yaml_file(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)
    
# read in all yaml files
token_addresses = read_yaml_file('token-addresses.yml')
ethereum_token_addresses = token_addresses['ethereum']
track_bridged_eth = read_yaml_file('track-bridged-eth.yml')

# Connect to Ethereum node
rpc_url = 'https://eth.llamarpc.com'
rpc_url = 'https://ethereum-rpc.publicnode.com'
rpc_url = 'https://eth-mainnet.public.blastapi.io'
rpc_url = 'https://mainnet.gateway.tenderly.co'
w3 = Web3(Web3.HTTPProvider(rpc_url))

# Function to get Ethereum balance for each address
def get_eth_balance(w3: Web3, address, at_block='latest'):
    try:
        balance = w3.eth.get_balance(Web3.to_checksum_address(address), block_identifier=at_block) / 10**18
    except Exception as e:
        print(f"Error retrieving balance for {address}: {e}")
    return balance

# Function to get ERC20 balance of a given coin & address
def get_erc20_balance_ethereum(w3: Web3, coin, address, at_block='latest'):
    contract_address = ethereum_token_addresses[coin]['contract']
    ABI = ethereum_token_addresses[coin]['abi']
    try:
        contract = w3.eth.contract(address=Web3.to_checksum_address(contract_address), abi=ABI)
        balance = contract.functions.balanceOf(Web3.to_checksum_address(address)).call(block_identifier=at_block) / 10**18
    except Exception as e:
        print(f"Error retrieving balance for {address}: {e}")
    return balance

# Function to get prices of wstETH, we assume 1stETH = 1 ETH
def get_ETH_wstETH_price(w3: Web3, at_block='latest'):
    address = ethereum_token_addresses['wstETH']['contract']
    abi = ethereum_token_addresses['wstETH']['abi']
    try:
        contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=abi)
        price = contract.functions.stEthPerToken().call(block_identifier=at_block) / 10**18
    except Exception as e:
        print(f"Error retrieving price for wstETH: {e}")
    return price

# Function to get price of mETH
def get_ETH_mETH_price(w3: Web3, at_block='latest'):
    address = ethereum_token_addresses['mETH']['staking_contract']
    abi = ethereum_token_addresses['mETH']['staking_abi']
    try:
        contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=abi)
        price = contract.functions.mETHToETH(10**18).call(block_identifier=at_block) / 10**18
    except Exception as e:
        print(f"Error retrieving price for mETH: {e}")
    return price

# Function to get price of pufETH
def get_ETH_pufETH_price(w3: Web3, at_block='latest'):
    address = ethereum_token_addresses['pufETH']['contract']
    abi = ethereum_token_addresses['pufETH']['abi']
    try:
        contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=abi)
        price = contract.functions.convertToAssets(10**18).call(block_identifier=at_block) / 10**18
    except Exception as e:
        print(f"Error retrieving price for pufETH: {e}")
    return price

# Function to get price of weETH, assuming 1 eETH = 1 ETH
def get_ETH_weETH_price(w3: Web3, at_block='latest'):
    address = ethereum_token_addresses['weETH']['contract']
    abi = ethereum_token_addresses['weETH']['abi']
    try:
        contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=abi)
        price = contract.functions.getEETHByWeETH(10**18).call(block_identifier=at_block) / 10**18
    except Exception as e:
        print(f"Error retrieving price for weETH: {e}")
    return price

# Function to get price of rETH
def get_ETH_rETH_price(w3: Web3, at_block='latest'):
    address = ethereum_token_addresses['rETH']['contract']
    abi = ethereum_token_addresses['rETH']['abi']
    try:
        contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=abi)
        price = contract.functions.getExchangeRate().call(block_identifier=at_block) / 10**18
    except Exception as e:
        print(f"Error retrieving price for rETH: {e}")
    return price

# Function to get price of cbETH
def get_ETH_cbETH_price(w3: Web3, at_block='latest'):
    address = ethereum_token_addresses['cbETH']['contract']
    abi = ethereum_token_addresses['cbETH']['abi']
    try:
        contract = w3.eth.contract(address=Web3.to_checksum_address(address), abi=abi)
        price = contract.functions.exchangeRate().call(block_identifier=at_block) / 10**18
    except Exception as e:
        print(f"Error retrieving price for cbETH: {e}")
    return price

# Function to get the block number of the first block of the day using binary search
def get_first_block_of_day(w3: Web3, target_date: datetime.date):
    # Convert the target date to a UNIX timestamp (00:00:00 of that day in UTC)
    start_of_day = datetime.datetime.combine(target_date, datetime.time(0, 0), tzinfo=datetime.timezone.utc)
    start_timestamp = int(start_of_day.timestamp())

    # Get the current block number (latest block)
    latest_block = w3.eth.get_block('latest')['number']

    # Perform binary search to find the block with the timestamp >= start_timestamp
    low = 0
    high = latest_block

    while low < high:
        mid = (low + high) // 2
        mid_block = w3.eth.get_block(mid)
        if mid_block['timestamp'] < start_timestamp:
            low = mid + 1
        else:
            high = mid

    # After the search, 'low' should be the first block of the day or a block at or after the target timestamp
    first_block_of_day = w3.eth.get_block(low)

    return first_block_of_day if first_block_of_day['timestamp'] >= start_timestamp else None

# Function to lookup the block number of a given date using block_timestamps.csv
def lookup_block_number(date: str):
    df = pd.read_csv('block_timestamps.csv')
    try:
        block_number = int(df.loc[df['date'] == date, 'block'].values[0])
    except:
        block_number = None
    return block_number

## Script to backfill block_timestamps.csv

In [2]:
# get the last date in the csv
df = pd.read_csv('block_timestamps.csv')
last_date = pd.to_datetime(df['date'].iloc[-1]).date()

# get current date
current_date = datetime.datetime.now().date()

while current_date > last_date:
    # get block number
    new_block = get_first_block_of_day(w3, last_date + datetime.timedelta(days=1))
    new_row = pd.DataFrame({'date': [str(last_date + datetime.timedelta(days=1))], 
                            'block': [new_block['number']], 
                            'block_timestamp': [new_block['timestamp']]})
    df = pd.concat([df, new_row], ignore_index=True)
    last_date = last_date + datetime.timedelta(days=1)
    # save to csv
    df.to_csv('block_timestamps.csv', index=False)

## Script to create folder structure and empty csv files

In [3]:
# get an empty dataframe
df = pd.read_csv('block_timestamps.csv')
df = df.drop(columns=['block', 'block_timestamp'])

# read in all yaml files
token_addresses = read_yaml_file('token-addresses.yml')
ethereum_token_addresses = token_addresses['ethereum']
track_bridged_eth = read_yaml_file('track-bridged-eth.yml')

# create holdings folder
if os.path.exists('holdings') == False:
    os.mkdir('holdings')

# create a dictionary for each entity
for entity in list(track_bridged_eth):
    if os.path.exists(f"holdings/{entity}") == False:
        os.mkdir(f"holdings/{entity}")

# create subfolder for each chain in entity
for entity in list(track_bridged_eth):
    for chain in list(track_bridged_eth[entity]):
        if os.path.exists(f"holdings/{entity}/{chain}") == False:
            os.mkdir(f"holdings/{entity}/{chain}")

# create a csv file for each token in each chain
for entity in list(track_bridged_eth):
    for chain in list(track_bridged_eth[entity]):
        for token in [list(item.keys())[0] for item in track_bridged_eth[entity][chain]]:
            if os.path.exists(f"holdings/{entity}/{chain}/{token}.csv") == False:
                df.to_csv(f"holdings/{entity}/{chain}/{token}.csv", index=False)

# create prices folder
if os.path.exists('prices') == False:
    os.mkdir('prices')

# create a folder for each chain in prices folder
for chain in list(token_addresses.keys()):
    if os.path.exists(f"prices/{chain}") == False:
        os.mkdir(f"prices/{chain}")

# create a csv file for each token in each chain
for chain in list(token_addresses.keys()):
    for token in list(token_addresses[chain]):
        if os.path.exists(f"prices/{chain}/{token}.csv") == False:
            df.to_csv(f"prices/{chain}/{token}.csv", index=False)


### extend the csv files with the new dates

for entity in list(track_bridged_eth):
    for chain in list(track_bridged_eth[entity]):
        for token in [list(item.keys())[0] for item in track_bridged_eth[entity][chain]]:
            df = pd.read_csv(f"holdings/{entity}/{chain}/{token}.csv")
            last_date = pd.to_datetime(df['date'].iloc[-1]).date()
            column_names = df.columns
            for date in pd.date_range(start=last_date + datetime.timedelta(days=1), end=current_date):
                new_row = pd.DataFrame({'date': [date.strftime('%Y-%m-%d')]}, columns=column_names)
                new_row.to_csv(f"holdings/{entity}/{chain}/{token}.csv", index=False, mode='a', header=False)

for chain in list(token_addresses.keys()):
    for token in list(token_addresses[chain]):
        df = pd.read_csv(f"prices/{chain}/{token}.csv")
        last_date = pd.to_datetime(df['date'].iloc[-1]).date()
        column_names = df.columns
        for date in pd.date_range(start=last_date + datetime.timedelta(days=1), end=current_date):
            new_row = pd.DataFrame({'date': [date.strftime('%Y-%m-%d')]}, columns=column_names)
            new_row.to_csv(f"prices/{chain}/{token}.csv", index=False, mode='a', header=False)

## Script to backfill holdings on Ethereum

In [5]:
# read in all yaml files
token_addresses = read_yaml_file('token-addresses.yml')
ethereum_token_addresses = token_addresses['ethereum']
track_bridged_eth = read_yaml_file('track-bridged-eth.yml')

for entity in track_bridged_eth.keys():

    print(f"Processing {entity}")

    for j in track_bridged_eth[entity]['ethereum']:

        # create a column for each address if not already in the dataframe
        asset = list(j.keys())[0]
        addresses = j[asset]
        df = pd.read_csv(f'holdings/{entity}/ethereum/{asset}.csv')
        for address in addresses[0]['address']:
            if address not in df.columns:
                df[address] = np.nan

        for address in addresses[0]['address']:
            # only process if the last row of the column is not yet filled
            if np.isnan(df[address].iloc[-1]) == False:
                print(f"Skipping {address} as already filled")
                pass
            else:
                print(f"Backfilling {address}")
                # iterate through each row from reverse until we get balance = 0
                for i in range(len(df)-1, -1, -1):
                    date = df['date'].iloc[i]
                    block = lookup_block_number(date)
                    #print(f"Processing {date}")
                    if asset == 'ETH':
                        balance = get_eth_balance(w3, address, block)
                    else:
                        balance = get_erc20_balance_ethereum(w3, asset, address, block)

                    if np.isnan(df.loc[i, address]):
                        df.loc[i, address] = balance
                        if i % 100 == 0:
                            df.to_csv(f"holdings/{entity}/ethereum/{asset}.csv", index=False)
                    else: # stop if we already have a value (full backfilled)
                        break

                    # stop also if balance is 0 = contract not yet used or deployed / no coins on it
                    #if balance == 0:
                    #    break

                # save to csv
                df.to_csv(f"holdings/{entity}/ethereum/{asset}.csv", index=False)


Processing Beacon_chain_deposits
Skipping 0x00000000219ab540356cbb839cbe05303d7705fa as already filled
Processing Arbitrum
Skipping 0x8315177ab297ba92a06054ce80a67ed4dbd7ed3a as already filled
Skipping 0x011B6E24FfB0B5f5fCc564cf4183C5BBBc96D515 as already filled
Skipping 0x0F25c1DC2a9922304f2eac71DCa9B07E310e8E5a as already filled
Processing Base
Skipping 0x49048044d57e1c92a77f79988d21fa8faf74e97e as already filled
Skipping 0x9de443AdC5A411E83F1878Ef24C3F52C61571e72 as already filled
Skipping 0x3154Cf16ccdb4C6d922629664174b904d80F2C35 as already filled
Processing Optimism
Skipping 0xbEb5Fc579115071764c7423A4f12eDde41f106Ed as already filled
Skipping 0x99C9fc46f92E8a1c0deC1b1747d010903E884bE1 as already filled
Skipping 0x76943C0D61395d8F2edF9060e1533529cAe05dE6 as already filled
Processing Scroll
Skipping 0x6774Bcbd5ceCeF1336b5300fb5186a12DDD8b367 as already filled
Skipping 0xA033Ff09f2da45f0e9ae495f525363722Df42b2a as already filled
Skipping 0x6625C6332c9F91F2D27c304E729B86db87A3f504 a

KeyboardInterrupt: 

In [108]:
120 % 100

20

## Script to backfill prices

In [28]:
# only works for assets on ethereum L1 as of now!

for chain in token_addresses:

    print(f"Processing {chain}")

    for token in token_addresses[chain]:

        # create a column for each address if not already in the dataframe
        df = pd.read_csv(f'prices/{chain}/{token}.csv')
        if 'price' not in df.columns:
            df['price'] = None
            df.to_csv(f"prices/{chain}/{token}.csv", index=False)

        # only process if the whole column is not yet filled
        if df['price'].isnull().all() == False:
            print(f"Skipping {token} prices as already filled")
            pass
        else:
            print(f"Backfilling {token}")
            # iterate through each row from reverse until we get price = 0
            for i in range(len(df)-1, -1, -1):
                date = df['date'].iloc[i]
                block = lookup_block_number(date)
                try:
                    if token == 'wstETH':
                        price = get_ETH_wstETH_price(w3, block)
                    elif token == 'mETH':
                        price = get_ETH_mETH_price(w3, block)
                    elif token == 'pufETH':
                        price = get_ETH_pufETH_price(w3, block)
                    elif token == 'weETH':
                        price = get_ETH_weETH_price(w3, block)
                    elif token == 'rETH':
                        price = get_ETH_rETH_price(w3, block)
                    elif token == 'cbETH':
                        price = get_ETH_cbETH_price(w3, block)
                    elif token in ['wETH', 'stETH', 'eETH']: # assuming 1 token = 1 ETH
                        price = 1
                    else:
                        print(f"Token {token} not supported")
                except Exception as e:
                    print(f"Error retrieving price for {token}: {e}")
                    print("Assuming date reached where contract not yet deployed or implementation changed.")
                    break
                df.loc[i, 'price'] = price

                # save to csv
                df.to_csv(f"prices/{chain}/{token}.csv", index=False)

                # stop if balance is 0 = contract not yet used
                if price == 0:
                    break

Processing ethereum
Skipping wETH prices as already filled
Skipping stETH prices as already filled
Skipping wstETH prices as already filled
Skipping mETH prices as already filled
Skipping cbETH prices as already filled
Skipping pufETH prices as already filled
Skipping eETH prices as already filled
Skipping weETH prices as already filled
Backfilling rETH
Error retrieving price for rETH: Could not decode contract function call to getExchangeRate with return data: b'', output_types: ['uint256']
Error retrieving price for rETH: cannot access local variable 'price' where it is not associated with a value
Assuming date reached where contract not yet deployed or implementation changed.


## Script to get the total supply over time ...? 

In [ ]:
# Etherscan is paid plan only
# Coingecko API is free but only estimate
# DUNE???

import requests

# Etherscan API key
ETHERSCAN_API_KEY = os.environ['ETHERSCAN_API_KEY']

# Function to get ETH total supply at a block
def get_eth_supply_at_block(block_number):
    url = f'https://api.etherscan.io/api?module=proxy&action=eth_getBlockByNumber&tag={hex(block_number)}&boolean=true&apikey={ETHERSCAN_API_KEY}'
    response = requests.get(url)
    data = response.json()

    if 'result' in data:
        block_reward = int(data['result']['blockReward'], 16)  # Block reward in Wei
        return block_reward / (10 ** 18)  # Convert Wei to ETH
    else:
        print(f"Error fetching ETH supply at block {block_number}: {data}")
        return None
    
